In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import minmax_scale

In [2]:
# #PATH
path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"
path_to_predict = "F:/178/Tugas Akhir/dataset/predict/"

# path_info = "C:/Users/USER/Downloads/Tugas Akhir/dataset/annotations/"
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/coba fitur/"
# path_to_experiment ="C:/Users/USER/Downloads/Tugas Akhir/dataset/experiment/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"
# path_to_predict = "C:/Users/USER/Downloads/Tugas Akhir/dataset/predict/"

In [3]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

def split(data,value):
    train_data, test_data, train_values, test_values = train_test_split(data,value,test_size=0.3,random_state=1000)
    
    return train_data, test_data, train_values, test_values

def training(data,values,clf):
    trained = clf.fit(data,values)
    
    return trained
    
def predict(test,true,clf):
    pred = clf.predict(test)
    df = pd.DataFrame(data=[pred,true])
    
    return pred,df

def score(pred,true):
    r2 = r2_score(true, pred)
    
    return r2

def crosval(clf,data,values):
    scores = cross_val_score(clf,data,values,scoring='r2',cv=10)
    
    return scores

def save_model(model,case,types):
    path = path_to_model+types+case+'.sav' # save the model
    joblib.dump(model, path)

def save_predict(df1,df2,case):
    path = path_to_predict+case+".xlsx" # save the model
    arousal = df1.T
    valence = df2.T
    merged = arousal.merge(valence, how='outer', left_index=True, right_index=True)
    merged.to_excel(path,header=None,index=None)

def norm(data):
    scaler.fit(data)
    normalized = scaler.transform(data)
    
    return normalized

In [4]:
kernels = RationalQuadratic(length_scale=1.0, alpha=1.0, 
                            length_scale_bounds=(1e-6, 1e6), 
                            alpha_bounds=(1e-6, 1e6))

# CASE 1

In [5]:
datafitur = "case1" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

#split arousal
train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar)
#split valence
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) 

(744, 26)


INITIAL MODEL

In [6]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Arousal

In [7]:
print("arousal:\t",gpr_a.kernel)
print("arousal:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
arousal:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [8]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [9]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [10]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0307, length_scale=4.12)
valence:	 RationalQuadratic(alpha=0.0311, length_scale=2)


AROUSAL

In [11]:
# df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.088238,0.235580,0.584982,-0.405384,-0.410139,-0.086915,-0.809901,0.261002,-0.443631,-0.091556,...,0.487067,1.177945e-01,0.001863,-0.155370,-0.057214,-0.004002,0.324561,0.356283,-0.228,-0.453356
1,0.352941,0.264706,0.411765,-0.558824,-0.441176,-0.411765,-0.588235,0.235294,-0.411765,0.352941,...,0.588235,-2.220446e-16,-0.382353,0.235294,-0.088235,-0.029412,-0.058824,0.382353,-0.500,-0.823529


R2 Score Arousal:	0.64


VALENCE

In [12]:
# df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.179347,0.324119,0.485976,-0.145413,-0.068155,0.106264,-0.321716,0.232469,-0.074331,0.044818,...,0.425284,0.331999,0.005170,-0.038969,-0.014026,0.051305,0.158851,-0.001306,-0.139625,-0.448638
1,-0.661538,0.476923,0.353846,-0.661538,0.046154,-0.507692,-0.076923,-0.169231,-0.230769,0.569231,...,0.538462,-0.015385,-0.723077,0.230769,-0.323077,-0.138462,0.507692,0.507692,-0.261538,-0.415385


R2 Score Valence: 0.29


REGRESSION

In [13]:
scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00064483, -0.0321948 ]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 3, 'warnflag': 2}
  ConvergenceWarning)
C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00063104, -0.00156853]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 2, 'warnflag': 2}
  ConvergenceWarning)


AROUSAL

In [14]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.39113875  0.47493296  0.51594495  0.51277248  0.66185402  0.56923713
  0.51420647  0.59929901  0.64047246  0.5246607 ]
R2 Score Valence:	0.54 (+/- 0.08)


VALENCE

In [15]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.08777192  0.03972039  0.02537876 -0.16568591  0.36585407  0.09138349
  0.31712252  0.26211364  0.27678954  0.11349062]
R2 Score Valence:	0.14 (+/- 0.15)


In [16]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 2

In [17]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 34)


INITIAL MODEL

In [18]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Valence

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Valence

In [19]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [20]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [21]:
# #Predict Arousal
pred_a,df2_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df2_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [22]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0296, length_scale=5.29)
valence:	 RationalQuadratic(alpha=0.0301, length_scale=3.23)


AROUSAL

In [23]:
# df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.088238,0.235580,0.584982,-0.405384,-0.410139,-0.086915,-0.809901,0.261002,-0.443631,-0.091556,...,0.487067,1.177945e-01,0.001863,-0.155370,-0.057214,-0.004002,0.324561,0.356283,-0.228,-0.453356
1,0.352941,0.264706,0.411765,-0.558824,-0.441176,-0.411765,-0.588235,0.235294,-0.411765,0.352941,...,0.588235,-2.220446e-16,-0.382353,0.235294,-0.088235,-0.029412,-0.058824,0.382353,-0.500,-0.823529


R2 Score Arousal:	0.67


VALENCE

In [24]:
# df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.179347,0.324119,0.485976,-0.145413,-0.068155,0.106264,-0.321716,0.232469,-0.074331,0.044818,...,0.425284,0.331999,0.005170,-0.038969,-0.014026,0.051305,0.158851,-0.001306,-0.139625,-0.448638
1,-0.661538,0.476923,0.353846,-0.661538,0.046154,-0.507692,-0.076923,-0.169231,-0.230769,0.569231,...,0.538462,-0.015385,-0.723077,0.230769,-0.323077,-0.138462,0.507692,0.507692,-0.261538,-0.415385


R2 Score Valence:	0.37


REGRESSION

In [25]:
scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [26]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.47786951  0.54044672  0.58834367  0.63571148  0.65205551  0.61552497
  0.63893716  0.65625517  0.59748882  0.58834416]
R2 Score Valence:	0.60 (+/- 0.05)


VALENCE

In [27]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.34296968  0.2963413   0.25693837  0.12841282  0.49458752  0.27202872
  0.46245318  0.35035195  0.3201791   0.17479126]
R2 Score Valence:	0.31 (+/- 0.11)


In [28]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 3

In [29]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 37)


INITAL MODEL

In [30]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Valence

In [31]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [32]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [33]:
# #Predict Arousal
pred_a,df2_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df2_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [34]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0282, length_scale=5.4)
valence:	 RationalQuadratic(alpha=0.0304, length_scale=3.4)


AROUSAL

In [35]:
# df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.088238,0.235580,0.584982,-0.405384,-0.410139,-0.086915,-0.809901,0.261002,-0.443631,-0.091556,...,0.487067,1.177945e-01,0.001863,-0.155370,-0.057214,-0.004002,0.324561,0.356283,-0.228,-0.453356
1,0.352941,0.264706,0.411765,-0.558824,-0.441176,-0.411765,-0.588235,0.235294,-0.411765,0.352941,...,0.588235,-2.220446e-16,-0.382353,0.235294,-0.088235,-0.029412,-0.058824,0.382353,-0.500,-0.823529


R2 Score Arousal:	0.68


VALENCE

In [36]:
# df3_v = df3_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.179347,0.324119,0.485976,-0.145413,-0.068155,0.106264,-0.321716,0.232469,-0.074331,0.044818,...,0.425284,0.331999,0.005170,-0.038969,-0.014026,0.051305,0.158851,-0.001306,-0.139625,-0.448638
1,-0.661538,0.476923,0.353846,-0.661538,0.046154,-0.507692,-0.076923,-0.169231,-0.230769,0.569231,...,0.538462,-0.015385,-0.723077,0.230769,-0.323077,-0.138462,0.507692,0.507692,-0.261538,-0.415385


R2 Score Valence:	0.38


REGRESSION

In [37]:
scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [38]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.50404329  0.52933545  0.55470409  0.63116308  0.65790515  0.63105683
  0.64902058  0.64100453  0.59219615  0.60790025]
R2 Score Valence:	0.60 (+/- 0.05)


VALENCE

In [39]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.34787595  0.30856516  0.25407681  0.12942156  0.51124481  0.26999648
  0.45645089  0.33966892  0.31398068  0.17234916]
R2 Score Valence:	0.31 (+/- 0.11)


In [40]:
save_predict(df_a,df_v,datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 4

In [41]:
datafitur =  "case4" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 61)


INITIAL MODEL

In [42]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Valence

In [43]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [44]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [45]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [46]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.037, length_scale=9.43)
valence:	 RationalQuadratic(alpha=0.0394, length_scale=6.13)


AROUSAL

In [47]:
# df4_a = df4_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.066998,0.157116,0.531436,-0.598410,-0.320814,-0.240503,-0.714524,0.303569,-0.279799,-0.027215,...,0.545318,1.269967e-01,-0.208889,-0.327222,-0.081239,0.046240,0.376124,0.478750,-0.315545,-0.496355
1,0.352941,0.264706,0.411765,-0.558824,-0.441176,-0.411765,-0.588235,0.235294,-0.411765,0.352941,...,0.588235,-2.220446e-16,-0.382353,0.235294,-0.088235,-0.029412,-0.058824,0.382353,-0.500000,-0.823529


R2 Score Arousal:	0.68


VALENCE

In [48]:
# df4_v = df4_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.150411,0.290213,0.548553,-0.166714,0.000528,-0.020325,-0.388268,0.339922,0.101775,0.059128,...,0.299177,0.112597,-0.230976,-0.245303,-0.113981,0.030735,0.041307,0.168819,-0.288231,-0.039562
1,-0.661538,0.476923,0.353846,-0.661538,0.046154,-0.507692,-0.076923,-0.169231,-0.230769,0.569231,...,0.538462,-0.015385,-0.723077,0.230769,-0.323077,-0.138462,0.507692,0.507692,-0.261538,-0.415385


R2 Score Valence:	0.38


REGRESSION

In [49]:
scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [50]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.44672873  0.5839231   0.61084799  0.53452106  0.65829641  0.66989345
  0.6430872   0.66133519  0.62616777  0.66062924]
R2 Score Valence:	0.61 (+/- 0.07)


VALENCE

In [51]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.31867443  0.35038908  0.23122918  0.05437017  0.4048447   0.31051334
  0.42644845  0.29673161  0.27565131  0.28379686]
R2 Score Valence:	0.30 (+/- 0.10)


In [52]:
save_predict(df_a,df_v,datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence